In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import re
import string
from nltk.corpus import stopwords
import string
from datetime import timedelta
import datetime as dt

In [2]:
df=pd.read_csv('reddit_wsb.csv',usecols=['title',"score","comms_num","body","timestamp"])
df.head(5)

,title,score,comms_num,body,timestamp
0,"It's not about the money, it's about sending a...",55,6,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,23,NaN,2021-01-28 21:32:10
2,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,74,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,156,NaN,2021-01-28 21:26:56


In [3]:
df.dtypes

title        object
score         int64
comms_num     int64
body         object
timestamp    object
dtype: object

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53187 entries, 0 to 53186
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      53187 non-null  object
 1   score      53187 non-null  int64 
 2   comms_num  53187 non-null  int64 
 3   body       24738 non-null  object
 4   timestamp  53187 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


In [5]:
duplicates = df.duplicated()
print(duplicates)
df[duplicates]
df.drop_duplicates()
df.shape

0        False
1        False
2        False
3        False
4        False
         ...  
53182    False
53183    False
53184    False
53185    False
53186    False
Length: 53187, dtype: bool


(53187, 5)

In [6]:
df=df[df['body'].notna()]

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date

In [12]:
df['title_count'] = df.title.apply(lambda x: len(str(x).split(' ')))

In [14]:
df.head(5)

,title,score,comms_num,body,timestamp,date,title_count
2,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,2021-01-28,3
6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,2021-01-28,7
7,THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,2021-01-28,4
10,"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,2021-01-28,12
12,"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,2021-01-28,17


In [15]:
df.describe()

,score,comms_num,title_count
count,24738.000000,24738.000000,24738.000000
mean,543.380306,401.836850,9.484882
std,3509.233280,3364.110139,7.025007
min,0.000000,0.000000,1.000000
25%,1.000000,2.000000,5.000000
50%,14.000000,14.000000,8.000000
75%,107.000000,58.000000,12.000000
max,111559.000000,93268.000000,64.000000
